# Parsing a torch model in MLIR Syntax

Those can already be generated by [Torch-MLIR](https://github.com/llvm/torch-mlir)!

Let's just parse it and print it for now:

In [ ]:
from torch.dialect import Torch
from xdsl.dialects.func import Func
from xdsl.dialects.builtin import Builtin
from xdsl.parser import Parser, Source
from xdsl.printer import Printer
from xdsl.ir import MLContext

context = MLContext()
context.register_dialect(Torch)
context.register_dialect(Func)
context.register_dialect(Builtin)

with open('examples/alexnet.mlir')as f:
    parser = Parser(context, f.read(), Source.MLIR, f.name)
    module = parser.parse_module()

printer = Printer()

printer.print(module)